In [ ]:
import pandas as pd
import json
from google.colab import files
from datetime import datetime
import numpy as np

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer)):
            return int(obj)
        if isinstance(obj, (np.floating)):
            return float(obj)
        if isinstance(obj, (datetime, pd.Timestamp)):
            return obj.isoformat()
        return super().default(obj)

def load_and_clean_data():
    """Handles file upload and data cleaning"""
    print("Please upload your Excel file:")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]

    # Load and clean data
    df = pd.read_excel(filename, header=3)
    df['EVENT DAY'] = pd.to_datetime(df['EVENT DAY'], errors='coerce')
    df['RESOLVED TIME'] = pd.to_datetime(df['RESOLVED TIME'], errors='coerce')
    df['CLIENT NAME'] = df['CLIENT NAME'].replace({'americanauae': 'UAE', 'americanaksa': 'KSA'})
    df['SOURCE NAME'] = df['SOURCE NAME'].str.strip()
    df['resolution_minutes'] = (df['RESOLVED TIME'] - df['EVENT DAY']).dt.total_seconds() / 60
    df['month'] = df['EVENT DAY'].dt.month_name()
    df['week_of_month'] = (df['EVENT DAY'].dt.day - 1) // 7 + 1

    # Filter valid resolutions
    valid_resolutions = df.dropna(subset=['RESOLVED TIME'])
    valid_resolutions = valid_resolutions[valid_resolutions['RESOLVED TIME'] >= valid_resolutions['EVENT DAY']]

    return valid_resolutions

def analyze_equipment_source(data):
    results = {
        "name_count": {
            "dsc": None,
            "asc": None
        },
        "in_month": {
            "dsc": [],
            "asc": []
        },
        "time_taken_to_resolve": {
            "dsc": None,
            "asc": None
        }
    }

    # 1. Name Count Analysis
    name_count = data['SOURCE NAME'].value_counts().reset_index()
    name_count.columns = ['Source name', 'Count']
    source_info = data.drop_duplicates('SOURCE NAME').set_index('SOURCE NAME')[['NAME', 'PATH']]
    name_count = name_count.join(source_info, on='Source name')

    results["name_count"]["dsc"] = name_count.sort_values('Count', ascending=False).to_dict('records')
    results["name_count"]["asc"] = name_count.sort_values('Count', ascending=True).to_dict('records')

    # 2. Monthly/Weekly Analysis (updated to match your requested format)
    month_analysis = data.groupby(['month', 'week_of_month', 'NAME']).agg({
        'SOURCE NAME': 'count',
        'PATH': lambda x: x.value_counts().index[0]
    }).reset_index()
    month_analysis.columns = ['Month', 'Week of Month', 'Alarm Type', 'Count', 'Most Common Path']

    # Get top alarms per month-week
    top_monthly = (month_analysis.sort_values(['Month', 'Week of Month', 'Count']),ascending=[True, True, False])

    # Format for JSON output
    for (month, week), group in top_monthly.groupby(['Month', 'Week of Month']):
        month_data = {
            'Month': month,
            'Week of Month': f"Week {week}",
            'Total Alarms': group['Count'].sum(),
            'Top Alarms': group[['Alarm Type', 'Count', 'Most Common Path']]
                          .head(3).to_dict('records')
        }
        results["in_month"]["dsc"].append(month_data)
        results["in_month"]["asc"].append(month_data)

    # Sort ascending by total alarms
    results["in_month"]["asc"].sort(key=lambda x: x['Total Alarms'])

    # 3. Resolution Time Analysis
    resolve_time = data.groupby('SOURCE NAME').agg({
        'resolution_minutes': 'mean',
        'NAME': 'first',
        'PATH': 'first'
    }).reset_index()
    resolve_time.columns = ['Source name', 'Time taken to resolve in minutes', 'Alarm Type', 'Path']

    results["time_taken_to_resolve"]["dsc"] = resolve_time.sort_values('Time taken to resolve in minutes', ascending=False).to_dict('records')
    results["time_taken_to_resolve"]["asc"] = resolve_time.sort_values('Time taken to resolve in minutes', ascending=True).to_dict('records')

    return results

def analyze_condition_name(data):
    conditions = {
        'door_open': 'Door Open',
        'extremely_high_temp': 'Extremely High Temperature',
        'high_temp': 'High Temperature',
        'low_temp': 'Low Temperature',
        'no_power': 'No power',
        'site_not_communicating': 'Site Not Communicating'
    }

    results = {}

    for cond_key, cond_name in conditions.items():
        cond_data = data[data['NAME'] == cond_name]
        if not cond_data.empty:
            # Count analysis
            count = cond_data['SOURCE NAME'].value_counts().reset_index()
            count.columns = ['Source name', 'Count']
            source_info = cond_data.drop_duplicates('SOURCE NAME').set_index('SOURCE NAME')[['PATH']]
            count = count.join(source_info, on='Source name')

            # Time taken analysis
            time_taken = cond_data.groupby('SOURCE NAME').agg({
                'resolution_minutes': 'mean',
                'PATH': 'first'
            }).reset_index()
            time_taken.columns = ['Source name', 'Time taken', 'Path']

            results[cond_key] = {
                'count': {
                    'dsc': count.sort_values('Count', ascending=False).to_dict('records'),
                    'asc': count.sort_values('Count', ascending=True).to_dict('records')
                },
                'time_taken': {
                    'dsc': time_taken.sort_values('Time taken', ascending=False).to_dict('records'),
                    'asc': time_taken.sort_values('Time taken', ascending=True).to_dict('records')
                }
            }
        else:
            results[cond_key] = "No data available for this condition"

    return results

def save_and_download(data, filename):
    """Save analysis results to JSON and download"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2, cls=CustomJSONEncoder)
    files.download(filename)
    print(f"✅ Downloaded: {filename}")

def main():
    data = load_and_clean_data()

    equipment_results = analyze_equipment_source(data)
    condition_results = analyze_condition_name(data)

    while True:
        print("\n🔍 MAIN MENU")
        print("1. Equipment Source Analysis")
        print("2. Condition Name Analysis")
        print("3. Export Full Results")
        print("4. Exit")

        main_choice = input("Select option: ")

        if main_choice == "1":
            print("\nSelect Equipment Analysis to Download:")
            print("1. Name Count (DSC)")
            print("2. Name Count (ASC)")
            print("3. Monthly/Weekly Analysis (DSC)")
            print("4. Monthly/Weekly Analysis (ASC)")
            print("5. Resolution Time (DSC)")
            print("6. Resolution Time (ASC)")
            print("7. Back")

            equip_choice = input("Select option: ")

            if equip_choice == "1":
                save_and_download(
                    equipment_results["name_count"]["dsc"],
                    "Equipment_Name_Count_DSC.json"
                )
            elif equip_choice == "2":
                save_and_download(
                    equipment_results["name_count"]["asc"],
                    "Equipment_Name_Count_ASC.json"
                )
            elif equip_choice == "3":
                save_and_download(
                    equipment_results["in_month"]["dsc"],
                    "Equipment_Monthly_Analysis_DSC.json"
                )
            elif equip_choice == "4":
                save_and_download(
                    equipment_results["in_month"]["asc"],
                    "Equipment_Monthly_Analysis_ASC.json"
                )
            elif equip_choice == "5":
                save_and_download(
                    equipment_results["time_taken_to_resolve"]["dsc"],
                    "Equipment_Resolution_Time_DSC.json"
                )
            elif equip_choice == "6":
                save_and_download(
                    equipment_results["time_taken_to_resolve"]["asc"],
                    "Equipment_Resolution_Time_ASC.json"
                )

        elif main_choice == "2":
            print("\nSelect Condition to Download:")
            print("1. Door Open")
            print("2. Extremely High Temperature")
            print("3. High Temperature")
            print("4. Low Temperature")
            print("5. No Power")
            print("6. Site Not Communicating")
            print("7. Back")

            cond_choice = input("Select condition: ")

            conditions = [
                ("1", "door_open"),
                ("2", "extremely_high_temp"),
                ("3", "high_temp"),
                ("4", "low_temp"),
                ("5", "no_power"),
                ("6", "site_not_communicating")
            ]

            selected_cond = None
            for num, cond in conditions:
                if cond_choice == num:
                    selected_cond = cond
                    break

            if selected_cond and selected_cond in condition_results:
                print(f"\nDownload Options for {selected_cond.replace('_', ' ').title()}:")
                print("1. Count Analysis (DSC)")
                print("2. Count Analysis (ASC)")
                print("3. Time Taken Analysis (DSC)")
                print("4. Time Taken Analysis (ASC)")
                print("5. Back")

                analysis_choice = input("Select option: ")

                if analysis_choice == "1":
                    save_and_download(
                        condition_results[selected_cond]["count"]["dsc"],
                        f"{selected_cond}_Count_DSC.json"
                    )
                elif analysis_choice == "2":
                    save_and_download(
                        condition_results[selected_cond]["count"]["asc"],
                        f"{selected_cond}_Count_ASC.json"
                    )
                elif analysis_choice == "3":
                    save_and_download(
                        condition_results[selected_cond]["time_taken"]["dsc"],
                        f"{selected_cond}_Time_Taken_DSC.json"
                    )
                elif analysis_choice == "4":
                    save_and_download(
                        condition_results[selected_cond]["time_taken"]["asc"],
                        f"{selected_cond}_Time_Taken_ASC.json"
                    )

        elif main_choice == "3":
            full_results = {
                "Equipment_Analysis": equipment_results,
                "Condition_Analysis": condition_results
            }
            save_and_download(full_results, "Full_Analysis_Results.json")

        elif main_choice == "4":
            print("Exiting program...")
            break

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-1-cb6de82fbe4f>, line 72)